In [11]:
import cv2
import pandas as pd
import numpy as np
import os
# function to read image
def read_image(filename : str)-> np.array:
    return cv2.imread(filename)

# converting image to grayscale image
def get_graycale(image:np.array):
    weights= [0.3, 0.58, 0.12]
    gray = np.dot(img[..., :3], weights)
    return np.round(gray).astype(np.int16)

img = read_image("data/cse-logo.png")
print(img.shape)


(100, 100, 3)


In [12]:

gray_image = get_graycale(img)
print("max_val",np.max(gray_image))
print("min_val",np.min(gray_image))
print("gray_image shape : ",gray_image.shape)

max_val 255
min_val 0
gray_image shape :  (100, 100)


In [14]:
def calculate_histogram(image:np.array):
    array = image.flatten()
    histogram = np.array([0]*256)
    for x in array:
        histogram[x]+=1
    return histogram
histogram =calculate_histogram(gray_image)
# print(histogram)
r_star =np.argmax(histogram)
print("r* = ",r_star)

r* =  255


In [10]:
t=10
a = np.where(np.abs(gray_image - r_star) <= 10, 1, 0).astype(np.uint8)
cv2.imshow('Binary Image', a * 255)  # Multiply by 255 to visualize
cv2.waitKey(8000)
cv2.destroyAllWindows()
print("t = ",t)


t =  10


In [9]:
#************** Q1 B************
#Nearest neighbour interpolation
def get_rounded(scale_factor:float,val:float)->int:
    if(scale_factor < 1):
        return int(val/scale_factor)
    else:
        return int(val*scale_factor)

def Nearest_neighbour_interpolation(image:np.array):
    image_shape = image.shape
    N = int(.25*max(image_shape[0],image_shape[1]))
    row_scale = image_shape[0]/N
    col_scale = image_shape[1]/N
    resized_image = np.zeros((N,N))
    
    for i in range(N):
        for j in range(N):
            i_old = int(i*row_scale)
            j_old = int(j*col_scale)
            resized_image[i,j] = image[i_old,j_old]

    return resized_image



In [15]:
nni_image =Nearest_neighbour_interpolation(a)
cv2.imshow('Binary Image', nni_image*255)  
cv2.waitKey(4000)
cv2.destroyAllWindows()
print("nn_interpolated_image_shape :",nni_image.shape)


nn_interpolated_image_shape : (25, 25)
